In [190]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time
import json

In [191]:
# Application Config
EnableColorStream = 0
ColorStreamResWidth = 1920
ColorStreamResHeight = 1080
ColorStreamFPS = 30

EnableInfraredStream = 1
InfraredStreamResWidth = 1280
InfraredStreamResHeight = 720
InfraredStreamFPS = 30

EnableDepthStream = 1
DepthStreamResWidth = 1280
DepthStreamResHeight = 720
DepthStreamFPS = 30

maxDistance = 1000      
capture = 0

In [192]:
# Configure depth and color streams
pipeline = rs.pipeline()
config = rs.config()

if EnableColorStream:
    config.enable_stream(
        rs.stream.color, ColorStreamResWidth, ColorStreamResHeight, rs.format.bgr8, ColorStreamFPS)

if EnableInfraredStream:
    config.enable_stream(
        rs.stream.infrared, InfraredStreamResWidth, InfraredStreamResHeight, rs.format.bgr8, InfraredStreamFPS)
    
if EnableDepthStream:
    config.enable_stream(
        rs.stream.depth, DepthStreamResWidth, DepthStreamResHeight, rs.format.z16, DepthStreamFPS)

# Start streaming
cfg = pipeline.start(config)
dev = cfg.get_device()
depth_sensor = dev.first_depth_sensor()
depth_sensor.set_option(rs.option.visual_preset, 4)
depth_sensor.set_option(rs.option.enable_auto_exposure, 1)

In [193]:
while True:
    # Wait for a coherent pair of frames: depth and color
    frames = pipeline.wait_for_frames()
    
    if EnableColorStream:
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue
            
    if EnableInfraredStream:
        infrared_frame = frames.first(rs.stream.infrared)
        if not infrared_frame:
            continue
    
    if EnableDepthStream:
        depth_frame = frames.get_depth_frame()
        if not depth_frame:
            continue
    
    
    # Convert images to numpy arrays
    if EnableColorStream:
        color_image = np.asanyarray(color_frame.get_data())
    
    if EnableInfraredStream:
        infrared_image = np.asanyarray(infrared_frame.get_data())
    
    if EnableDepthStream:
        depth_image = np.asanyarray(depth_frame.get_data())
    
    # colorize depth image
    minNum, maxNum, minLoc, maxLoc = cv2.minMaxLoc(depth_image)

    if maxNum > maxDistance:
        maxNum = maxDistance
    
    scaleAlpha = 255 / maxNum
    depth_image = cv2.convertScaleAbs(depth_image, None, scaleAlpha, 0)
    depth_colormap = cv2.applyColorMap(depth_image, cv2.COLORMAP_JET)
    
    temp = depth_sensor.get_option(rs.option.projector_temperature)
    print("proj: " + str(temp))
    temp = depth_sensor.get_option(rs.option.asic_temperature)
    print("asic: " + str(temp))

    # Show images
    if EnableColorStream:
        cv2.imshow('Color image', color_image)
        
    if EnableInfraredStream:
        cv2.imshow('Infrared image', infrared_image)
    
    if EnableDepthStream:
        cv2.imshow('Depth image', depth_colormap)
    
    # Screen capture
    if capture:
        if EnableColorStream:
            fileName = "../imageCapture/color_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, color_image, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
        if EnableDepthStream:
            fileName = "../imageCapture/depth_" + time.strftime("%Y-%m-%d_%H%M%S-", time.localtime()) + '.png'
            cv2.imwrite(fileName, depth_colormap, [int(cv2.IMWRITE_PNG_COMPRESSION), 0])
        
        capture = 0
    
    # Keyboard command
    getKey = cv2.waitKey(10) & 0xFF
    if getKey is ord('c') or getKey is ord('C'):
        capture = 1
    elif getKey is ord('q') or getKey is ord('Q'):
        break

proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0
asic: 70.0
proj: 58.0

In [194]:
# Stop streaming
cv2.destroyAllWindows()
pipeline.stop()